## Data Analysis de un Smartwatch con Python

El conjunto de datos que estoy utilizando para el análisis de datos de Smartwatch está disponible públicamente en este repositorio. Este conjunto de datos se recopiló inicialmente de 30 usuarias del reloj inteligente Fitbit. 

Ahora comenzaré la tarea de Análisis de datos de Smartwatch importando e instalando las bibliotecas de Python necesarias y el conjunto de datos:

In [1]:
pip install plotly==5.8.1

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install statsmodels

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

### Importamos la data

In [4]:
data = pd.read_csv('./data/dailyActivity.csv')
print(data.head())

           Id ActivityDate  TotalSteps  TotalDistance  TrackerDistance  \
0  1503960366    4/12/2016       13162           8.50             8.50   
1  1503960366    4/13/2016       10735           6.97             6.97   
2  1503960366    4/14/2016       10460           6.74             6.74   
3  1503960366    4/15/2016        9762           6.28             6.28   
4  1503960366    4/16/2016       12669           8.16             8.16   

   LoggedActivitiesDistance  VeryActiveDistance  ModeratelyActiveDistance  \
0                       0.0                1.88                      0.55   
1                       0.0                1.57                      0.69   
2                       0.0                2.44                      0.40   
3                       0.0                2.14                      1.26   
4                       0.0                2.71                      0.41   

   LightActiveDistance  SedentaryActiveDistance  VeryActiveMinutes  \
0                 6.06

Antes de seguir adelante, revisamos que este conjunto de datos no tenga algún valor nulo:

In [9]:
print(data.isnull().sum())


Id                          0
ActivityDate                0
TotalSteps                  0
TotalDistance               0
TrackerDistance             0
LoggedActivitiesDistance    0
VeryActiveDistance          0
ModeratelyActiveDistance    0
LightActiveDistance         0
SedentaryActiveDistance     0
VeryActiveMinutes           0
FairlyActiveMinutes         0
LightlyActiveMinutes        0
SedentaryMinutes            0
Calories                    0
TotalMinutes                0
dtype: int64


Por lo tanto, el conjunto de datos no tiene ningún valor nulo. Revisamos la información sobre las columnas del conjunto de datos:

In [5]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 940 entries, 0 to 939
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Id                        940 non-null    int64  
 1   ActivityDate              940 non-null    object 
 2   TotalSteps                940 non-null    int64  
 3   TotalDistance             940 non-null    float64
 4   TrackerDistance           940 non-null    float64
 5   LoggedActivitiesDistance  940 non-null    float64
 6   VeryActiveDistance        940 non-null    float64
 7   ModeratelyActiveDistance  940 non-null    float64
 8   LightActiveDistance       940 non-null    float64
 9   SedentaryActiveDistance   940 non-null    float64
 10  VeryActiveMinutes         940 non-null    int64  
 11  FairlyActiveMinutes       940 non-null    int64  
 12  LightlyActiveMinutes      940 non-null    int64  
 13  SedentaryMinutes          940 non-null    int64  
 14  Calories  

La columna que contiene la fecha del registro es un objeto. Es posible que necesitemos usar fechas en nuestro análisis, así que convirtamos esta columna en una columna de fecha y hora:

In [10]:
# Changing datatype of ActivityDate
data["ActivityDate"] = pd.to_datetime(data["ActivityDate"], 
                                      format="%m/%d/%Y")
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 940 entries, 0 to 939
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Id                        940 non-null    int64         
 1   ActivityDate              940 non-null    datetime64[ns]
 2   TotalSteps                940 non-null    int64         
 3   TotalDistance             940 non-null    float64       
 4   TrackerDistance           940 non-null    float64       
 5   LoggedActivitiesDistance  940 non-null    float64       
 6   VeryActiveDistance        940 non-null    float64       
 7   ModeratelyActiveDistance  940 non-null    float64       
 8   LightActiveDistance       940 non-null    float64       
 9   SedentaryActiveDistance   940 non-null    float64       
 10  VeryActiveMinutes         940 non-null    int64         
 11  FairlyActiveMinutes       940 non-null    int64         
 12  LightlyActiveMinutes  

Observando todas las columnas; verás información sobre minutos muy activos, bastante activos, ligeramente activos y sedentarios en el conjunto de datos. Combinemos todas estas columnas como minutos totales antes de avanzar:

In [11]:
data["TotalMinutes"] = data["VeryActiveMinutes"] + data["FairlyActiveMinutes"] + data["LightlyActiveMinutes"] + data["SedentaryMinutes"]
print(data["TotalMinutes"].sample(5))

670    1440
168    1440
376     742
156    1440
258    1018
Name: TotalMinutes, dtype: int64


Ahora revisamos las estadísticas descriptivas del conjunto de datos:

In [12]:
print(data.describe())

                 Id    TotalSteps  TotalDistance  TrackerDistance  \
count  9.400000e+02    940.000000     940.000000       940.000000   
mean   4.855407e+09   7637.910638       5.489702         5.475351   
std    2.424805e+09   5087.150742       3.924606         3.907276   
min    1.503960e+09      0.000000       0.000000         0.000000   
25%    2.320127e+09   3789.750000       2.620000         2.620000   
50%    4.445115e+09   7405.500000       5.245000         5.245000   
75%    6.962181e+09  10727.000000       7.712500         7.710000   
max    8.877689e+09  36019.000000      28.030001        28.030001   

       LoggedActivitiesDistance  VeryActiveDistance  ModeratelyActiveDistance  \
count                940.000000          940.000000                940.000000   
mean                   0.108171            1.502681                  0.567543   
std                    0.619897            2.658941                  0.883580   
min                    0.000000            0.000000   

Analicemos los datos⌚️ del smartwatch
El conjunto de datos tiene una columna "Calorías"; contiene los datos sobre el número de calorías quemadas en un día.  
Verificamos la relación entre las calorías quemadas y el total de pasos caminados en un día:

In [13]:
figure = px.scatter(data_frame = data, x="Calories",
                    y="TotalSteps", size="VeryActiveMinutes", 
                    trendline="ols", 
                    title="Relationship between Calories & Total Steps")
figure.show()

Puedes ver que existe una relación lineal entre el número total de pasos y el número de calorías quemadas en un día. Ahora veamos el número total promedio de minutos activos en un día:

In [14]:
label = ["Very Active Minutes", "Fairly Active Minutes", 
         "Lightly Active Minutes", "Inactive Minutes"]
counts = data[["VeryActiveMinutes", "FairlyActiveMinutes", 
               "LightlyActiveMinutes", "SedentaryMinutes"]].mean()
colors = ['gold','lightgreen', "pink", "blue"]

fig = go.Figure(data=[go.Pie(labels=label, values=counts)])
fig.update_layout(title_text='Total Active Minutes')
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=30,
                  marker=dict(colors=colors, line=dict(color='black', width=3)))
fig.show()

Observaciones:  
* 81,3% del total de minutos inactivos en un día
* 15.8% de minutos ligeramente activos en un día
* En promedio, solo 21 minutos (1,74%) fueron muy activos.
* y 1,11% (13 minutos) de minutos bastante activos en un día  

Transformamos el tipo de datos de la columna ActivityDate en la columna datetime anterior. Usémoslo para encontrar los días de la semana de los registros y agregar una nueva columna a este conjunto de datos como "Día":

In [15]:
data["Day"] = data["ActivityDate"].dt.day_name()
print(data["Day"].head())

0      Tuesday
1    Wednesday
2     Thursday
3       Friday
4     Saturday
Name: Day, dtype: object


Ahora listemos los minutos muy activos, bastante activos y ligeramente activos en cada día de la semana:

In [16]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=data["Day"],
    y=data["VeryActiveMinutes"],
    name='Very Active',
    marker_color='purple'
))
fig.add_trace(go.Bar(
    x=data["Day"],
    y=data["FairlyActiveMinutes"],
    name='Fairly Active',
    marker_color='green'
))
fig.add_trace(go.Bar(
    x=data["Day"],
    y=data["LightlyActiveMinutes"],
    name='Lightly Active',
    marker_color='pink'
))
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

Ahora a la cantidad de minutos inactivos en cada día de la semana:

In [17]:
day = data["Day"].value_counts()
label = day.index
counts = data["SedentaryMinutes"]
colors = ['gold','lightgreen', "pink", "blue", "skyblue", "cyan", "orange"]

fig = go.Figure(data=[go.Pie(labels=label, values=counts)])
fig.update_layout(title_text='Inactive Minutes Daily')
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=30,
                  marker=dict(colors=colors, line=dict(color='black', width=3)))
fig.show()

Por lo tanto, el jueves es el día más inactivo de acuerdo con el estilo de vida de todos los individuos en el conjunto de datos. Ahora revisamos la cantidad de calorías quemadas en cada día de la semana:

In [18]:
calories = data["Day"].value_counts()
label = calories.index
counts = data["Calories"]
colors = ['gold','lightgreen', "pink", "blue", "skyblue", "cyan", "orange"]

fig = go.Figure(data=[go.Pie(labels=label, values=counts)])
fig.update_layout(title_text='Calories Burned Daily')
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=30,
                  marker=dict(colors=colors, line=dict(color='black', width=3)))
fig.show()

: 

El martes es, por lo tanto, uno de los días más activos para todas las personas en el conjunto de datos, ya que el mayor número de calorías se quemó los martes.

Así es como puede analizar los datos del reloj inteligente utilizando el lenguaje de programación Python. Hay mucho más que puede hacer con este conjunto de datos. También puede usarlo para predecir la cantidad de calorías quemadas en un día.

Así es como puede analizar los datos recopilados por un reloj inteligente sobre el estado físico utilizando Python. Los relojes inteligentes son los preferidos por las personas a las que les gusta cuidar su estado físico. El análisis de los datos recopilados sobre su estado físico es uno de los casos de uso de la ciencia de datos en la atención médica. Espero que te haya gustado este artículo sobre análisis de datos de Smartwatch usando Python. 